# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [25]:
import findspark
findspark.init('/usr/local/spark/spark-3.1.2-bin-hadoop3.2')
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql.window import Window
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.types import IntegerType

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 

In [7]:
spark = SparkSession \
    .builder \
    .appName("Wrangling Data_quiz") \
    .getOrCreate()

In [10]:
path = "sparkify_log_small.json"
df = spark.read.json(path)

In [11]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Question 1

Which page did user id "" (empty string) NOT visit?

In [21]:
# TODO: write your code to answer question 1

# filter for users with blank user id
guest_pages = df.filter(df.userId == '')\
    .select(col('page'))\
    .alias('guest_pages')\
    .dropDuplicates()

# get a list of possible pages that could be visited
all_pages = df.select('page').dropDuplicates()

# find values in all_pages that are not in guest_pages
# these are the pages that the guest user did not go to
for i in set(all_pages.collect()) - set(guest_pages.collect()):
    print(i.page)

Error
Save Settings
Downgrade
Upgrade
NextSong
Submit Upgrade
Logout
Submit Downgrade
Settings


# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [22]:
# TODO: use this space to explore the behavior of the user with an empty string

# Answer: Users in guest mode, meaning not logged in or unregistered users

# filter for users with blank user id
guest_pages = df.filter(df.userId == '')\
    .select(col('page'))\
    .alias('guest_pages')\
    .dropDuplicates()

# these are the pages that the guest user go to
for i in set(guest_pages.collect()):
    print(i.page)

About
Home
Help
Login


# Question 3

How many female users do we have in the data set?

In [24]:
# TODO: write your code to answer question 3

df.filter(df.gender == 'F')\
    .select('userID', 'gender')\
    .dropDuplicates()\
    .count()

462

# Question 4

How many songs were played from the most played artist?

In [35]:
# TODO: write your code to answer question 4
df.filter(df.page == 'NextSong')\
    .select('Artist')\
    .groupBy('Artist')\
    .agg({'Artist':'count'})\
    .withColumnRenamed('count(Artist)', 'Artist_count')\
    .sort(desc('Artist_count'))\
    .show(1)

+--------+------------+
|  Artist|Artist_count|
+--------+------------+
|Coldplay|          83|
+--------+------------+
only showing top 1 row



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [39]:
# TODO: write your code to answer question 5

function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())

user_window = Window.partitionBy('userID') \
    .orderBy(desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

cusum = df.filter((df.page == 'NextSong') | (df.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(col('page'))) \
    .withColumn('period', Fsum('homevisit').over(user_window))

cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'})\
    .withColumnRenamed('avg(count(period))', 'AvgPlayingSongs')\
    .show()

+-----------------+
|  AvgPlayingSongs|
+-----------------+
|6.898347107438017|
+-----------------+

